

# <font size="+3"><span style='color:#2994ff'> **P7 - Implémentez un modèle de scoring** </span></font>


<a id='LOADING_LIBRARIES'></a>

---

---

<font size="+1"> **LOADING THE LIBRARIES** </font>

---

In [25]:

# ----------------
# Packages Update
# ----------------
# !pip install --upgrade pip
# !pip install pycodestyle
# !pip install flake8 pycodestyle_magic

# General libraries
import sys
import os
import glob
import pandas as pd
import numpy as np
import csv
import time
import datetime
import tools_dataframe
import tools_preprocessing
import pickle



# Data visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns



# Validation code PEP8
# %load_ext pycodestyle_magic
# %pycodestyle_on

# Warnings
# import warnings
# from warnings import simplefilter
# warnings.filterwarnings(action='once')
# simplefilter(action='ignore', category=FutureWarning)
# simplefilter(action='ignore', category=ValueError)
# simplefilter(action='ignore', category=UserWarning)

# Versions
print('Version of used libraries :')

print('Python                : ' + sys.version)
print('NumPy                 : ' + np.version.full_version)
print('Pandas                : ' + pd.__version__)
print('Dataframe tools      : ' + tools_dataframe.__version__)
now = datetime.datetime.now().isoformat()
print('Launched on : ' + now)

Version of used libraries :
Python                : 3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0]
NumPy                 : 1.21.6
Pandas                : 1.5.3
Dataframe tools      : 0.0.0
Launched on : 2023-06-09T18:33:42.280888



<a id='USED_PARAMETERS'></a>


<br>


---
---

<font size="+1"> **PARAMETERS USED IN THIS NOTEBOOK** </font>

---


In [2]:
# Parameters used in this notebook

seed = 84

palette4 = ["#253d85", "#618576", "#cba87d", "#faec4d"]

palette5 = ["#253d85", "#618576", "#9595b1", "#cba87d", "#faec4d"]

palette6 = ["#253d85", "#618576", "#9595b1", "#cba87d", "#dcd2a3", "#faec4d"]

In [3]:
dico_pd_option = {
    'display.max_rows': 400,
    'display.max_column': 200,
    'display.width': 300,
    'display.precision': 4,
    'display.max_colwidth': 1000,
    'mode.chained_assignment': None
}
for key, value in dico_pd_option.items():
    pd.set_option(key, value)


<a id='USED_FUNCTIONS'></a>


<br>


---
---

<font size="+1"> **FUNCTIONS USED IN THIS NOTEBOOK** </font>

---



<font size="+3"><span style='color:#2994ff'> **P7 - Implémentez un modèle de scoring** </span></font>


<center><img src="./img/logo_projet.png" style="height: 250px;"/></center>

**Prêt à dépenser** wants to implement a **credit scoring tool to calculate the probability of a customer repaying their loan**, and then **classify the application as either granted or refused credit**. It therefore wants to develop a classification algorithm based on a variety of data sources (behavioural data, data from other financial institutions, etc.).

This notebook includes the **pre-processing** of data sets and the **feature engineering**:


- application_train/set.csv,

- office.csv**,

- office_balance.csv**,

- credit_card_balance.csv**,

- installments_payments.csv**,

- POS_CASH_balance.csv**,

- previous_application.csv


In particular:


- Cleaning** :

    - memory optimisation** by changing the data type. 

    - correction of **outliers**
    
    - correction of duplicated data

    - imputation** of missing values

- Feature engineering**: in this section, we will start by using the knowledge and insights gained from exploratory data analysis to obtain good sets of variables using feature engineering. Note that most of the variables generated are based on domain knowledge and experimentation. Without good sets of variables, machine learning algorithms cannot produce effective results. We will therefore have to find ingenious ways of doing feature engineering so that the model makes better decisions. 

- Aggregation**: with main data from Home Crédit Group.


***

Sources

***

The aggregation functions have been taken from the notebook: [Source](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering/output) by adapting the signature to pass the prefix of the column names and to be able to pass a dictionary of the statistics you want to add.


## <font color = '#0085dd'>**Table of content**</font>


[Librairies loading](#LOADING_LIBRARIES)<br>

[Functions used in this notebook](#USED_FUNCTIONS)<br>

---

[**Datasets**](#datasets)
 * [Description](#datasets_description)
   * [orders_dataset](#orders_dataset)
   * [customers_dataset](#customers_dataset)
   * [order_items](#order_items)
   * [products_dataset](#products_dataset)
   * [product_category_name_translation](#product_category_name_translation)   
   * [sellers_dataset](#sellers_dataset)
   * [order_payments](#order_payments)
   * [order_reviews](#order_reviews)
   * [geolocation_dataset](#geolocation_dataset)<br> 
<br>
 * [Columns preparation](#columns_preparation)
   * [Zip codes centroids calculation](#centroid_zipCode)
   * [Product category check and update](#cat_products)   
   
   
[**Datasets joining**](#datasets_joining)
 * [Data description](#data_description)
 * [Evaluation of missing values](#missing_values)  
 * [Column filling analysis](#column_fill)   
 * [Row filling analysis](#row_filling)  
 * [Features pre-selection](#features_preSelection)

[**Features engineering**](#features_engineering)
 * [Customers spatial distribution](#customers_spatial_distribution)
 * [RFM features](#RFM_features)  
 * [Products](#products)   
 * [Orders](#Orders)  
 * [Dates](#dates)
 * [Joining customers information](#join_customers_datasets)
 * [Features analysis](#features_analysis)  

[**Dataset for segmentation**](#segmentation_dataset)
<br>

---


<a id='datasets_loading'></a>

---
---

# <span style='background:#2994ff'><span style='color:white'>**Datasets loading** </span></span>


In [4]:
# Define the folder containing the files with the project data
P7_source = "/home/raquelsp/Documents/Openclassrooms/P7_implementez_modele_scoring/P7_travail/p7_source"

os.chdir(P7_source)

In [5]:
# -----------------------------
# Files upload:
# -----------------------------
application_train = pd.read_csv('application_train.csv', low_memory=False,
                            encoding='utf-8')
application_test = pd.read_csv('application_test.csv', low_memory=False,
                           encoding='utf-8')
bureau = pd.read_csv('bureau.csv', low_memory=False, encoding='utf-8')

bureau_balance = pd.read_csv('bureau_balance.csv', low_memory=False,
                             encoding='utf-8')
previous_application = pd.read_csv('previous_application.csv',
                                   low_memory=False, encoding='utf-8')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv',
                               low_memory=False, encoding='utf-8')
installments_payments = pd.read_csv('installments_payments.csv',
                                    low_memory=False, encoding='utf-8')
cc_balance = pd.read_csv('credit_card_balance.csv',
                         low_memory=False, encoding='utf-8')

<a id='datasets_loading'></a>

---
---

# <span style='background:#2994ff'><span style='color:white'>**Train and test datasets** </span></span>


| File | Description |
| --- | --- |
| <p style='text-align: justify;'>**application_train.csv** <br> **application_test.csv**</p> | <ul style='text-align: justify;'><li>The main table, divided into two files for Train (with TARGET) and Test (without TARGET).</li> <li>Static data for all applications.</li><li>A line represents a loan in our data sample.</li></ul> |

<a id='application_train_dataset'></a>

## <span style='background:#0085dd'><span style='color:white'>Dataset application_train/test</span></span>

In [33]:
# --------------
# Dataset size
# --------------
nRow, nVar = application_train.shape
print(f'The application_train dataset contains {nRow} rows and {nVar} variables.')
print()
nRow, nVar = application_test.shape
print(f'The application_test dataset contains {nRow} rows and {nVar} variables.')

The application_train dataset contains 307511 rows and 112 variables.

The application_test dataset contains 48744 rows and 121 variables.


In [35]:
# Continous variables
cols_num_train = application_train.select_dtypes(include=[np.number])\
                                    .columns.to_list()
cols_num_test = application_test.select_dtypes(include=[np.number])\
                                    .columns.to_list()

In [36]:
# Categorical variables
cols_cat_train = application_train.select_dtypes(exclude=[np.number])\
                                    .columns.to_list()
cols_cat_test = application_test.select_dtypes(exclude=[np.number])\
                                    .columns.to_list()

In [39]:
# ----------------------------------------------------
# Memory improvement by reducing the size of objects
# ----------------------------------------------------
print("application_train")
application_train =\
    tools_preprocessing.reduce_mem_usage(application_train, verbose=True)

application_train
-------------------------------------------------------------------------------
Memory usage of dataframe is: 82.11 MB
Memory usage after optimization is : 82.11 MB
Reduction of 0.0%
-------------------------------------------------------------------------------


In [40]:
# ----------------------------------------------------
# Memory improvement by reducing the size of objects
# ----------------------------------------------------
print("application_test")
application_test =\
    tools_preprocessing.reduce_mem_usage(application_test, verbose=True)

application_test
-------------------------------------------------------------------------------
Memory usage of dataframe is: 14.60 MB
Memory usage after optimization is : 14.60 MB
Reduction of 0.0%
-------------------------------------------------------------------------------


In [43]:
# Manual changes : REGION_RATING_CLIENT contains 1, 2 and 3
# ==> object
application_train['REGION_RATING_CLIENT'] = \
    application_train['REGION_RATING_CLIENT'].astype('object')
# Manual changes : : REGION_RATING_CLIENT_W_CITY contains
# ==> 1, 2 ou 3 object
application_train['REGION_RATING_CLIENT_W_CITY'] = \
    application_train['REGION_RATING_CLIENT_W_CITY'].astype('object')

In [44]:
# # Manual changes : REGION_RATING_CLIENT contains 1, 2 and 3
# ==> object
application_test['REGION_RATING_CLIENT'] = \
    application_test['REGION_RATING_CLIENT'].astype('object')
# Manual changes : : REGION_RATING_CLIENT_W_CITY contains
# ==> 1, 2 ou 3 object
application_test['REGION_RATING_CLIENT_W_CITY'] = \
    application_test['REGION_RATING_CLIENT_W_CITY'].astype('object')

In [45]:
# EMERGENCYSTATE_MODE contains Yes/No ==> transform to 1/0
# %pycodestyle_off
dico_emergency = {'Yes' : 0,
                  'No' : 1,
                  np.nan : 0}
tools_preprocessing.transl_values(application_train,
                                  'EMERGENCYSTATE_MODE', dico_emergency)
application_train['EMERGENCYSTATE_MODE'] = \
    application_train['EMERGENCYSTATE_MODE'].astype('int8')
# %pycodestyle_on

In [49]:
# EMERGENCYSTATE_MODE contains Y/N ==> transformer en 1/0
# %pycodestyle_off
dico_emergency = {'Yes' : 0,
                  'No' : 1,
                  np.nan : 0}
tools_preprocessing.transl_values(application_test,
                                      'EMERGENCYSTATE_MODE', dico_emergency)
application_test['EMERGENCYSTATE_MODE'] = \
    application_test['EMERGENCYSTATE_MODE'].astype('int8')
# %pycodestyle_on

In [47]:
# FLAG_OWN_CAR contains Y/N ==> transformer en 1/0
# %pycodestyle_off
dico_owncar = {'Y' : 0,
               'N' : 1,
               np.nan : 0}
tools_preprocessing.transl_values(application_train, 'FLAG_OWN_CAR',
                                  dico_owncar)
application_train['FLAG_OWN_CAR'] = \
    application_train['FLAG_OWN_CAR'].astype('int8')
application_train['FLAG_OWN_CAR'].unique()
# %pycodestyle_on

array([1, 0], dtype=int8)

In [50]:
# FLAG_OWN_CAR contains Y/N ==> transformer en 1/0
# %pycodestyle_off
dico_owncar = {'Y' : 0,
               'N' : 1,
               np.nan : 0}
tools_preprocessing.transl_values(application_test, 'FLAG_OWN_CAR',
                                      dico_owncar)
application_test['FLAG_OWN_CAR'] = \
    application_test['FLAG_OWN_CAR'].astype('int8')
# %pycodestyle_on

In [13]:
# FLAG_OWN_REALTY contains Y/N ==> transformer en 1/0
# %pycodestyle_off
dico_ownreal = {'Y' : 0,
                'N' : 1,
                np.nan : 0}
tools_preprocessing.transl_values(application_train, 'FLAG_OWN_REALTY',
                                  dico_ownreal)
application_train['FLAG_OWN_REALTY'] = \
    application_train['FLAG_OWN_REALTY'].astype('int8')
application_train['FLAG_OWN_REALTY'].unique()
# %pycodestyle_on

array([0, 1], dtype=int8)

In [51]:
# FLAG_OWN_REALTY contains Y/N ==> transformer en 1/0
# %pycodestyle_off
dico_ownreal = {'Y' : 0,
                'N' : 1,
                np.nan : 0}
tools_preprocessing.transl_values(application_test, 'FLAG_OWN_REALTY',
                                      dico_ownreal)
application_test['FLAG_OWN_REALTY'] = \
    application_test['FLAG_OWN_REALTY'].astype('int8')
# %pycodestyle_on

<a id='application_train_outliers'></a>

---
### <span style='background:#0075bc'><span style='color:white'>Application_train outliers</span></span>

Correction of outliers detected during the EDA.

**DAYS_EMPLOYED**

From the 90th percentile, the value is 365243 days, i.e. 1000 years!

In [52]:
# Number of rows with errors
print("application_train")
nb_err = \
    application_train['DAYS_EMPLOYED'][application_train['DAYS_EMPLOYED']
                                       == 365243].count()
pourc_err = round((nb_err*100)/application_train.shape[0], 2)
print(f'Number of errors DAYS_EMPLOYED : {nb_err} ({pourc_err}%)')

application_train
Number of errors DAYS_EMPLOYED : 0 (0.0%)


In [53]:
# Correction
application_train['DAYS_EMPLOYED'][application_train['DAYS_EMPLOYED']
                                   == 365243] = np.nan

In [54]:
# Number of rows with errors
print("application_test")
nb_err = \
    application_test['DAYS_EMPLOYED'][application_test['DAYS_EMPLOYED']
                                       == 365243].count()
pourc_err = round((nb_err*100)/application_test.shape[0], 2)
print(f'Number of errors DAYS_EMPLOYED : {nb_err} ({pourc_err}%)')

application_test
Number of errors DAYS_EMPLOYED : 9274 (19.03%)


In [55]:
# Correction
application_test['DAYS_EMPLOYED'][application_test['DAYS_EMPLOYED']
                                   == 365243] = np.nan

<a id='application_train_missing_values'></a>

---
### <span style='background:#0075bc'><span style='color:white'>Application_train missing values</span></span>

In [16]:
# Valeurs manquantes du dataframe
df_nan_applitrain = tools_dataframe.get_missing_values(application_train,
                                                       True, False, True)

Missing values : 9062084 NaN for 37516342 data (24.16 %)
-------------------------------------------------------------
Number and % of missing values by variable



,Number of missing values,% of missing values
COMMONAREA_MODE,214865,69.870000
COMMONAREA_AVG,214865,69.870000
COMMONAREA_MEDI,214865,69.870000
NONLIVINGAPARTMENTS_MEDI,213514,69.430000
NONLIVINGAPARTMENTS_AVG,213514,69.430000
NONLIVINGAPARTMENTS_MODE,213514,69.430000
FONDKAPREMONT_MODE,210295,68.390000
LIVINGAPARTMENTS_MODE,210199,68.350000
LIVINGAPARTMENTS_AVG,210199,68.350000
LIVINGAPARTMENTS_MEDI,210199,68.350000



* From the important variables identified during the exploratory analysis to distinguish non-defaulters from defaulters, the FLOORSMIN_AVG variable has the highest number of missing values (67.85%).
* We set the threshold for deleting variables with many missing values at 68%.

In [56]:
# Valeurs manquantes du dataframe
df_nan_applitest = tools_dataframe.get_missing_values(application_test,
                                                       True, False, True)

Missing values : 1391484 NaN for 5898024 data (23.59 %)
-------------------------------------------------------------
Number and % of missing values by variable



,Number of missing values,% of missing values
COMMONAREA_MEDI,33495,68.720000
COMMONAREA_MODE,33495,68.720000
COMMONAREA_AVG,33495,68.720000
NONLIVINGAPARTMENTS_MODE,33347,68.410000
NONLIVINGAPARTMENTS_AVG,33347,68.410000
NONLIVINGAPARTMENTS_MEDI,33347,68.410000
FONDKAPREMONT_MODE,32797,67.280000
LIVINGAPARTMENTS_MEDI,32780,67.250000
LIVINGAPARTMENTS_AVG,32780,67.250000
LIVINGAPARTMENTS_MODE,32780,67.250000



* From the important variables identified during the exploratory analysis to distinguish non-defaulters from defaulters, the FLOORSMIN_AVG variable has the highest number of missing values (66.61%).
* We set the threshold for deleting variables with many missing values at 67%.

In [17]:
# List of variables with more than 68% of missing values
cols_nan_remov_train = \
    df_nan_applitrain[df_nan_applitrain['% of missing values'] > 68] \
    .index.to_list()
print(f'Number of variables to remove : {len(cols_nan_remov_train)}')
cols_nan_remov_train

Number of variables to remove : 10


['COMMONAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'COMMONAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'COMMONAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'FONDKAPREMONT_MODE']

In [57]:
# List of variables with more than 67% of missing values
cols_nan_remov_test = \
    df_nan_applitest[df_nan_applitest['% of missing values'] > 67] \
    .index.to_list()
print(f'Number of variables to remove : {len(cols_nan_remov_test)}')
cols_nan_remov_test

Number of variables to remove : 10


['COMMONAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'COMMONAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'COMMONAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'FONDKAPREMONT_MODE']

In [18]:
# Remove with % of nan over the threshold > 68%
application_train.drop(columns=cols_nan_remov, inplace=True)

# Categorical variables
cols_cat = application_train.select_dtypes(exclude=[np.number]).columns \
    .to_list()
# Continuous variables
cols_num_train = application_train.select_dtypes(include=[np.number]).columns \
    .to_list()

# Size : number of rows/columns
nRow, nVar = application_train.shape
print(f'The dataset application_train contains {nRow} rows and {nVar} variables.')

The dataset contains 307511 rows and 112 variables.


In [58]:
# Remove with % of nan over the threshold > 67%
application_test.drop(columns=cols_nan_remov_test, inplace=True)

# Categorical variables
cols_cat_test = application_test.select_dtypes(exclude=[np.number]).columns \
    .to_list()
# Continuous variables
cols_num_train = application_test.select_dtypes(include=[np.number]).columns \
    .to_list()

# Size : number of rows/columns
nRow, nVar = application_test.shape
print(f'The dataset application_test contains {nRow} rows and {nVar} variables.')

The dataset application_test contains 48744 rows and 111 variables.


<a id='application_train_assign_missing_values'></a>

---
<span style='background:#0075bc'><span style='color:white'>Application_train assignment missing values</span></span>

* During the exploratory analysis, it was observed that values that were not provided could give indications. The applicant may have withheld information to avoid having his application refused.
* The EDA also revealed the existence of many outliers.
* The **missing values** of the **continuous variables** will be therefore imputed by the **median** of each of these variables.
* The **missing values** of the **categorical variables** will be therefore imputed by the **mode** of each of these variables.

**Continuous variables**

In [19]:
nb_nan_median = application_train[cols_num_train].isna().sum().sum()
print(f'Number of nan before median assignment : {nb_nan_median}')
application_train.fillna(application_train[cols_num_train].median(), inplace=True)
# Validation
nb_nan_median = application_train[cols_num_train].isna().sum().sum()
print(f'Number of nan after median assignment : {nb_nan_median}')

Number of nan before median assignment : 6527734
Number of nan after median assignment : 0


**Categorical variables**

In [20]:
nb_nan_cat = application_train[cols_cat].isna().sum().sum()
print(f'Number of nan before mode assignment : {nb_nan_cat}')
for var in cols_cat:
    mode = application_train[var].mode()[0]
    application_train[var].fillna(mode, inplace=True)
nb_nan_cat = application_train[cols_cat].isna().sum().sum()
print(f'Number of nan after mode assignment  : {nb_nan_cat}')

Number of nan before mode assignment : 408321
Number of nan after mode assignment  : 0


In [26]:
# Validation
print(f'Number of nan in application_train : {application_train.isna().sum().sum()}')
# Saving application_train
path_sav_appltrain_wonan = \
    '/home/raquelsp/Documents/Openclassrooms/P7_implementez_modele_scoring/P7_travail/P7_scoring_credit/preprocessing/application_train_wo_nan.pickle'
with open(path_sav_appltrain_wonan, 'wb') as f:
    pickle.dump(application_train, f, pickle.HIGHEST_PROTOCOL)

Number of nan in application_train : 0


<a id='application_train_duplicated_data'></a>

---
### <span style='background:#0075bc'><span style='color:white'>Application_train duplicated data</span></span>

In [32]:
print("Duplicated data in application_train: ",\
      application_train.loc[application_train.duplicated()].shape[0])

Duplicated data in application_train:  0


# END